In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import kagglehub

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 15
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-4

In [3]:
path = kagglehub.dataset_download("ankit1743/skyview-an-aerial-landscape-dataset")
DATA_DIR = os.path.join(path, "Aerial_Landscapes")

In [4]:
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(),
    transforms.ToTensor(),
])
test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
])

In [5]:
dataset = datasets.ImageFolder(DATA_DIR, transform=train_transforms)
class_names = dataset.classes

train_len = int(0.8 * len(dataset))
test_len = len(dataset) - train_len
train_data, test_data = random_split(dataset, [train_len, test_len])

train_data.dataset.transform = train_transforms
test_data.dataset.transform = test_transforms

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [6]:
def get_custom_resnet50(num_classes=15, freeze_layers=True):
    model = models.resnet50(pretrained=True)

    if freeze_layers:
        for name, param in model.named_parameters():
            if "layer4" not in name and "fc" not in name:
                param.requires_grad = False

    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128, num_classes)
    )

    return model

In [7]:
def train_model(model, train_loader, test_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

    for epoch in range(EPOCHS):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1} | Loss: {running_loss / len(train_loader):.4f}")
        evaluate_model(model, test_loader)

In [8]:
def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, targets in loader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(targets.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

In [9]:
def main():
    model = get_custom_resnet50(num_classes=NUM_CLASSES, freeze_layers=True).to(DEVICE)
    train_model(model, train_loader, test_loader)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 220MB/s]
Epoch 1/10: 100%|██████████| 300/300 [01:26<00:00,  3.46it/s]


Epoch 1 | Loss: 0.6865

Classification Report:
              precision    recall  f1-score   support

 Agriculture       0.97      0.98      0.98       145
     Airport       0.96      0.98      0.97       181
       Beach       0.96      0.96      0.96       162
        City       0.99      0.96      0.97       158
      Desert       0.92      0.99      0.95       159
      Forest       0.95      0.97      0.96       154
   Grassland       0.93      0.97      0.95       160
     Highway       0.99      0.94      0.97       144
        Lake       0.95      0.97      0.96       158
    Mountain       0.98      0.88      0.93       165
     Parking       0.98      0.98      0.98       161
        Port       0.98      0.97      0.97       159
     Railway       0.95      0.97      0.96       173
 Residential       0.99      0.99      0.99       166
       River       0.97      0.94      0.96       155

    accuracy                           0.96      2400
   macro avg       0.97      0.96

Epoch 2/10: 100%|██████████| 300/300 [00:46<00:00,  6.45it/s]


Epoch 2 | Loss: 0.1536

Classification Report:
              precision    recall  f1-score   support

 Agriculture       0.97      0.98      0.98       145
     Airport       0.96      0.97      0.96       181
       Beach       0.98      0.98      0.98       162
        City       0.95      0.99      0.97       158
      Desert       0.91      0.99      0.95       159
      Forest       0.96      0.99      0.98       154
   Grassland       0.99      0.97      0.98       160
     Highway       0.92      0.98      0.95       144
        Lake       0.99      0.96      0.97       158
    Mountain       0.98      0.90      0.94       165
     Parking       0.99      0.98      0.99       161
        Port       0.98      0.96      0.97       159
     Railway       0.98      0.92      0.95       173
 Residential       0.99      0.98      0.98       166
       River       0.96      0.97      0.97       155

    accuracy                           0.97      2400
   macro avg       0.97      0.97

Epoch 3/10: 100%|██████████| 300/300 [00:47<00:00,  6.31it/s]


...

Epoch 8 | Loss: 0.0411

Classification Report:
              precision    recall  f1-score   support

 Agriculture       0.98      0.98      0.98       145
     Airport       0.94      0.99      0.96       181
       Beach       0.99      0.90      0.94       162
        City       0.99      0.97      0.98       158
      Desert       0.88      0.99      0.93       159
      Forest       0.99      0.99      0.99       154
   Grassland       0.98      0.99      0.98       160
     Highway       0.99      0.96      0.98       144
        Lake       0.90      0.99      0.94       158
    Mountain       0.98      0.89      0.93       165
     Parking       0.97      0.99      0.98       161
        Port       0.95      0.95      0.95       159
     Railway       0.98      0.96      0.97       173
 Residential       0.99      0.98      0.98       166
       River       0.99      0.95      0.97       155

    accuracy                           0.96      2400
   macro avg       0.97      0.97

Epoch 9/10: 100%|██████████| 300/300 [00:47<00:00,  6.37it/s]


Epoch 9 | Loss: 0.0322

Classification Report:
              precision    recall  f1-score   support

 Agriculture       0.95      0.99      0.97       145
     Airport       0.98      0.97      0.98       181
       Beach       0.97      0.98      0.98       162
        City       0.96      0.97      0.97       158
      Desert       0.95      0.99      0.97       159
      Forest       0.93      0.99      0.96       154
   Grassland       0.98      0.98      0.98       160
     Highway       0.99      0.98      0.99       144
        Lake       0.96      0.97      0.96       158
    Mountain       0.99      0.90      0.94       165
     Parking       0.95      0.99      0.97       161
        Port       0.99      0.96      0.98       159
     Railway       0.98      0.91      0.94       173
 Residential       1.00      0.98      0.99       166
       River       0.96      0.97      0.96       155

    accuracy                           0.97      2400
   macro avg       0.97      0.97

Epoch 10/10: 100%|██████████| 300/300 [00:46<00:00,  6.48it/s]


Epoch 10 | Loss: 0.0293

Classification Report:
              precision    recall  f1-score   support

 Agriculture       0.98      0.99      0.98       145
     Airport       0.97      0.95      0.96       181
       Beach       0.99      0.98      0.99       162
        City       0.98      0.97      0.98       158
      Desert       0.96      0.99      0.98       159
      Forest       0.96      0.99      0.98       154
   Grassland       0.98      0.99      0.98       160
     Highway       0.99      0.97      0.98       144
        Lake       0.99      0.97      0.98       158
    Mountain       0.99      0.94      0.96       165
     Parking       0.95      1.00      0.98       161
        Port       0.97      0.96      0.97       159
     Railway       0.95      0.98      0.97       173
 Residential       0.99      0.99      0.99       166
       River       0.98      0.96      0.97       155

    accuracy                           0.98      2400
   macro avg       0.98      0.9